## Credentials

In [4]:
!mkdir ~ /.aws

In [5]:
% % writefile ~ /.aws / credentials

[default]
aws_access_key_id =
aws_secret_access_key =

Writing /home/cyprien/.aws/credentials


In [6]:
% % writefile ~ /.aws / config

[default]
region = eu - west - 3

Writing /home/cyprien/.aws/config


## Export to s3

In [17]:
import os

# Set the directory you want to search
folder_path = '../../basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/'

# Set the file extensions you want to search for
extensions = ["_preprocessed.mp4", "_preprocessed.jsonl", "_preprocessed_annotations.jsonl"]

files = []

# Iterate through the files in the directory
for file in os.listdir(folder_path):
    # Check if the file name ends with any of the specified extensions
    if any(file.endswith(ext) for ext in extensions):
        # If the file name ends with any of the extensions, print the file name
        files.append(file)

len(files), len(set(files))

(12654, 12654)

In [21]:
from tqdm import tqdm
import sys

sys.path.append("../")
from utils.invalid_files import pp_video, pp_actions, pp_annotations

from concurrent.futures import ThreadPoolExecutor
import boto3
import os


def export_files_to_s3(bucket_name, s3_prefix, local_folder, file_extensions):
    # Create an S3 client
    s3 = boto3.client('s3')

    files = []

    for file_name in os.listdir(local_folder):
        if any([file_name.endswith(file_extension) for file_extension in file_extensions]):
            files.append(file_name)

    # Create a ThreadPoolExecutor with a certain number of threads
    with ThreadPoolExecutor(max_workers=50) as executor:
        # Iterate through all the files in the folder
        for file_name in tqdm(files[:]):
            # Check if the file has the desired extension
            if any([file_name.endswith(ext) for ext in file_extensions]):
                # Construct the full path to the file
                file_path = os.path.join(folder_path, file_name)
                # Construct the full S3 object key (including the path within the bucket)
                s3_key = os.path.join(s3_prefix, file_name)
                # Check if the file already exists in the S3 bucket
            try:
                s3_object = s3.head_object(Bucket=bucket_name, Key=s3_key)
                # If the file exists, check if the size of the local file is the same as the size of the S3 object
                if os.path.getsize(file_path) == s3_object['ContentLength']:
                    # If the sizes are the same, skip the file
                    continue
            except s3.exceptions.ClientError as e:
                # If the file does not exist, or if there is any other error, catch the exception and proceed with the upload
                pass

            # Submit a task to the executor to upload the file to S3
            executor.submit(s3.upload_file, file_path, bucket_name, s3_key)


bucket_name = 'basalt-neurips'
folder_path = '../../basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/'
s3_prefix = 'basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/'

export_files_to_s3(bucket_name, s3_prefix, folder_path, [pp_video, pp_actions, pp_annotations])

100%|██████████| 12654/12654 [07:10<00:00, 29.39it/s]


## Import from s3

In [3]:
import sys
import boto3

sys.path.append('../')
from typing import List
from utils.io_s3_tools import get_s3_files, get_s3_files_ends_with

s3 = boto3.client('s3')
objects = get_s3_files_ends_with(s3, "basalt-neurips", "basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/", "mp4")
len(objects)

4218

In [38]:
# s3.download_file("basalt-neurips", "basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/cheeky-cornflower-setter-00df0a566647-20220717-135630_preprocessed.jsonl", "tmp.jsonl")

In [1]:
ok

NameError: name 'ok' is not defined

In [ ]:

import sys
import boto3

sys.path.append('../')
from utils.io_s3_tools import get_s3_files, get_s3_files_ends_with

from typing import List
from utils.io_s3_tools import get_s3_files, get_s3_files_ends_with
from concurrent.futures import as_completed
from tqdm import tqdm
import sys
from utils.io_s3_tools import threaded_download_from_s3

sys.path.append("../")

from utils.invalid_files import pp_video, pp_actions, pp_annotations
import boto3
import os

import boto3
import os
from concurrent.futures import ThreadPoolExecutor


def download_folder_from_s3(bucket_name, s3_prefix, local_folder, file_extensions, threads=30):
    # Create an S3 client
    s3 = boto3.client('s3')

    objects = [j for i in [get_s3_files_ends_with(s3, "basalt-neurips", s3_prefix, ext) for ext in file_extensions] for
               j in i]
    print(len(objects))
    # Create a ThreadPoolExecutor with a certain number of threads

    threaded_download_from_s3(s3, bucket_name=bucket_name, files=objects, s3_prefix=s3_prefix,
                              local_folder=local_folder, threads=threads,
                              show_progress=True,
                              log_errors=True)


download_folder_from_s3(bucket_name='basalt-neurips',
                        s3_prefix='basalt_neurips_data/MineRLBasaltMakeWaterfall-v0/',
                        local_folder='../',
                        file_extensions=[pp_video, pp_actions, pp_annotations])

/home/cyprien/.conda/envs/mc/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


12654


  0%|          | 1/12654 [00:00<3:06:26,  1.13it/s][ERROR] utils.io_s3_tools: Error downloading file: {'_condition': <Condition(<unlocked _thread.RLock object owner=0 count=0 at 0x7f1b926e4060>, 0)>, '_state': 'FINISHED', '_result': None, '_exception': FileNotFoundError(2, 'No such file or directory'), '_waiters': [], '_done_callbacks': []}
